In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from tensorflow.keras import layers, losses

import warnings
warnings.filterwarnings('ignore')

## Análisis de los datos

In [2]:
df = pd.read_csv("data/movimientos.csv", sep=';', encoding='utf-8')
df.shape

(15432, 5)

In [3]:
df.head()

,fc_oper,channel,amount,operations_number,category_name
0,190101,Offline,14013.98,106,Mantenimiento y Reparaciones Auto
1,190101,Online,30343.54,1607,Gaming
2,190101,Offline,319937.82,19080,Supermercados y Pequeño Comercio
3,190101,Online,24172.25,2395,Contenidos digitales y media
4,190101,Online,27734.58,315,Mantenimiento y Reparaciones Auto


In [4]:
df['fc_oper'].max() - 10000

191004

In [5]:
#df_copy = df.copy()
#df_copy['fc_oper'] = df_copy['fc_oper'].astype(str)
#print(df_copy.dtypes)
#df_copy

In [6]:
df.isnull().sum()

fc_oper              0
channel              0
amount               0
operations_number    0
category_name        0
dtype: int64

No hay valores nulos en ninguna columna.

A continuación filtro los datos por fecha agrupados por categoría y canal, generando dos DataFrames distintos: uno con los datos correspondientes a los meses de pandemia y otro con los datos correspondientes a los mismos meses del año pasado.

In [7]:
af_covid_df = df[df['fc_oper'] > 200300].groupby(by=['category_name', 'channel']).agg({'amount':'sum', 'operations_number':'sum'})
af_covid_df.columns = af_covid_df.columns.values + "_After_Covid"

In [8]:
bf_covid_df = df[(df['fc_oper'] > 190300) & (df['fc_oper'] < (df['fc_oper'].max() - 10000))].groupby(by=['category_name', 'channel']).agg({'amount':'sum', 'operations_number':'sum'})
bf_covid_df.columns = bf_covid_df.columns.values + "_Before_Covid"

Junto los datos generados en un mismo DataFrame.

In [9]:
new_df = pd.concat([bf_covid_df, af_covid_df], axis=1)
new_df = new_df.reset_index(level=['category_name','channel'])
new_df.head()

,category_name,channel,amount_Before_Covid,operations_number_Before_Covid,amount_After_Covid,operations_number_After_Covid
0,Alojamiento,Offline,3.159033e+08,2452640,1.505664e+08,1420743
1,Alojamiento,Online,1.185686e+08,564535,6.285560e+07,275168
2,Bar y Discotecas,Offline,2.717977e+07,1267054,1.214223e+07,591495
3,Bar y Discotecas,Online,8.844461e+05,18238,2.888120e+05,8983
4,Bricolaje y Jardinería,Offline,2.604138e+08,4263670,2.665164e+08,4784789


Y añado dos nuevas columnas con la información que hace referencia a la diferencia de gasto y de número de operaciones entre el periodo de pandemia y el mismo periodo del año pasado.

In [10]:
new_df['amount_Diff'] = new_df['amount_After_Covid'] - new_df['amount_Before_Covid']
new_df['ops_Diff'] = new_df['operations_number_After_Covid'] - new_df['operations_number_Before_Covid']
new_df[['category_name', 'channel', 'amount_Diff', 'ops_Diff']].head()

,category_name,channel,amount_Diff,ops_Diff
0,Alojamiento,Offline,-1.653369e+08,-1031897
1,Alojamiento,Online,-5.571299e+07,-289367
2,Bar y Discotecas,Offline,-1.503754e+07,-675559
3,Bar y Discotecas,Online,-5.956341e+05,-9255
4,Bricolaje y Jardinería,Offline,6.102658e+06,521119


In [11]:
new_df.sort_values(by='amount_Diff')[['category_name', 'channel', 'amount_Diff', 'ops_Diff']]

,category_name,channel,amount_Diff,ops_Diff
18,Restaurantes y Cafeterías,Offline,-2.714966e+08,-6915700
6,Combustible y Recarga,Offline,-2.102938e+08,-4339931
0,Alojamiento,Offline,-1.653369e+08,-1031897
1,Alojamiento,Online,-5.571299e+07,-289367
14,Mantenimiento y Reparaciones Auto,Offline,-4.210168e+07,-188963
16,Peluquería y Barbería,Offline,-3.249806e+07,-819306
2,Bar y Discotecas,Offline,-1.503754e+07,-675559
3,Bar y Discotecas,Online,-5.956341e+05,-9255
7,Combustible y Recarga,Online,-8.087096e+04,-2760
23,"Tabaco, Alcohol y Otros",Online,2.250931e+05,3155


Después de ordenar los valores por la diferencia de gasto, se puede observar como las tres categorías/sectores más afectados negativamente son la venta *offline* de 'Restaurantes y Cafeterías', 'Combustible y Recarga', 'Alojamiento'.

Mientras que la venta, tanto *online* como *offline* de 'Supermercados y Pequeño Comercio' y la venta *offline* de 'Tabaco, Alcohol y Otros' son las tres categorías/sectores que han tenido un mayor beneficio en comparación con el mismo periodo del año pasado.

In [12]:
new_df.groupby('category_name').agg({'amount_Diff':'sum', 'ops_Diff':'sum'}).sort_values(by='amount_Diff')

,amount_Diff,ops_Diff
category_name,,
Restaurantes y Cafeterías,-2.662239e+08,-6644998
Alojamiento,-2.210499e+08,-1321264
Combustible y Recarga,-2.103746e+08,-4342691
Mantenimiento y Reparaciones Auto,-3.789421e+07,-169674
Peluquería y Barbería,-3.132723e+07,-804382
Bar y Discotecas,-1.563317e+07,-684814
Contenidos digitales y media,1.859196e+06,123311
Gaming,9.654294e+06,691998
Bricolaje y Jardinería,2.170252e+07,662770


Sin tener en cuenta el canal, la tabla de arriba muestra el impacto, ordenado de más negativo a más positivo, agrupando los datos por categoría y ordenando por la diferencia de gasto.

In [13]:
new_df[new_df['category_name'] == 'Contenidos digitales y media'][['category_name', 'channel', 'amount_Diff', 'ops_Diff']]

,category_name,channel,amount_Diff,ops_Diff
8,Contenidos digitales y media,Offline,307882.64,-31372
9,Contenidos digitales y media,Online,1551313.35,154683


En lo que se refiere a la categoría 'Contenidos digitales y media', se puede apreciar claramente el impacto de la pandemia, en el sentido de que la gente sale menos de casa y compra más por internet, ya que en este caso se ha reducido el número de operaciones *offline* y al mismo tiempo se ha aumentado en gran medida el número de operaciones *online* (154683 más con respecto al mismo periodo del año pasado).

En términos globales, el sector ha recaudado 1.859.196€ más que el año pasado en este periodo, por lo que podemos decir que el impacto ha sido positivo.

## Ejercicio 2

In [14]:
train_alojamiento = df[(df['fc_oper'] < 200900) & (df['category_name'] == 'Alojamiento')]
test_alojamiento = df[((200900 < df['fc_oper']) & (df['fc_oper'] < 201000)) & (df['category_name'] == 'Alojamiento')]

train_gaming = df[(df['fc_oper'] < 200900) & (df['category_name'] == 'Gaming')]
test_gaming = df[((200900 < df['fc_oper']) & (df['fc_oper'] < 201000)) & (df['category_name'] == 'Gaming')]

In [15]:
train_alojamiento = train_alojamiento.groupby('fc_oper').agg({'amount':'sum', 'operations_number':'sum'})
train_gaming = train_gaming.groupby('fc_oper').agg({'amount':'sum', 'operations_number':'sum'})

test_alojamiento = test_alojamiento.groupby('fc_oper').agg({'amount':'sum', 'operations_number':'sum'})
test_gaming = test_gaming.groupby('fc_oper').agg({'amount':'sum', 'operations_number':'sum'})

train_alojamiento.head()

,amount,operations_number
fc_oper,,
190101,2029809.24,10012
190102,1875531.28,9582
190103,1346418.57,8896
190104,1482828.56,9275
190105,1165678.76,7830


In [16]:
def create_model():
    new_model = tf.keras.Sequential([
        layers.Dense(16, input_dim=2, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation="linear")
    ])

    new_model.compile(loss='mse', optimizer='adam')

    return new_model

In [ ]:
X_train = train_alojamiento.reset_index(level=['fc_oper'])[['fc_oper', 'operations_number']].values#.reshape(-1, 1)
y_train = train_alojamiento['amount'].values.reshape(-1, 1)

X_test = test_alojamiento.reset_index(level=['fc_oper'])[['fc_oper', 'operations_number']].values#.reshape(-1, 1)
y_test = test_alojamiento['amount'].values.reshape(-1, 1)

model = create_model()
history = model.fit(x=X_train, y=y_train, epochs=1000)

#reg = LinearRegression().fit(X_train, y_train)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(loss, accuracy)

In [ ]:


y_pred = reg.predict(X_test)

plt.scatter(X_test, y_test, color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)
plt.show()